In [ ]:
""""
OANDAデータ→修正BBデータ変換

""""

%matplotlib inline

import glob
import csv

ORG_PRICE_DATA_DIR = "../money/data" 

PRICE_DATA_FILE = "./data.tsv"

HEADER = ["highBid", "closeAsk", "lowBid", "lowAsk", "openAsk", "complete", "highAsk", "time", "closeBid", "openBid", "volume"]

files = glob.glob(ORG_PRICE_DATA_DIR + "/USD_JPY/*/*/*")
# files = glob.glob(ORG_PRICE_DATA_DIR + "/USD_JPY/2015/01/0*")
price_idx = HEADER.index("closeAsk")
time_idx = HEADER.index("time")
with open(PRICE_DATA_FILE, "w") as ofp:
    for f in files:
        date = f.split("/")
        year = date[-3]
        month = date[-2]
        day = date[-1].split(".")[0]
        writer = csv.writer(ofp, delimiter = "\t", lineterminator = "\n")
        with open(f, "r") as ifp:
            reader = csv.reader(ifp, delimiter = "\t")
            for line in reader:
                writer.writerow(["-".join([year, month, day]) + " " + line[time_idx], line[price_idx]])

print("Success")


In [ ]:
""""
修正BB係数算出

入力データ
　フォーマット：TSV
　形式　　　　：日時(yyyy-mm-dd hh:MM:ss), 価格

""""

import numpy as np
import csv
from datetime import datetime as dt

DATA = "./data.tsv"
OUTPUT_FILE = "./threshold.tsv"

MOV_AVG_SPAN = 13
STD_SPAN = 20

DATE_IDX = 0
PRICE_IDX = 1

COEF_WIDTH = 5.0
COEF_STEP = 1.0

def is_calc_coef(prev_date, date):
    if prev_date != None and prev_date.day != date.day:     # 日
    # if prev_date != None and prev_date.month != date.month:     # 月
        return True
    else:
        return False

def calculate_coef(data, coef_range):
    max_profit = 0
    result_max_coef = None
    result_min_coef = None
    for max_coef in coef_range:
        for min_coef in coef_range:
            position = 0.0
            asset = 0.0
            for d in data:
                date = d["date"]
                price = d["price"]
                mavg = d["mavg"]
                std = d["std"]
                is_update = False
                
                min_th = mavg - std * min_coef
                max_th = mavg + std * max_coef
                # print("Min TH\t" + str(min_th) + "\t" + "MAX TH\t" + str(max_th))
                # print("Mean Avg\t" + str(mavg) + "\t" + "STD\t" + str(std) + "\t"
                #        "Min Coef\t" + str(min_coef) + "\t" + "Max Coef\t" + str(max_coef))
                if price < min_th:
                    position += 1
                    asset = asset - price
                    is_update = True
                if price > max_th:
                    position -= 1
                    asset = asset + price
                    is_update = True

                    
                # print("Mean Avg\t" + str(mavg) + "\t" + "STD\t" + str(std) + "\t"
                #          "Min Coef\t" + str(min_coef) + "\t" + "Max Coef\t" + str(max_coef))
                # # # print(mavg - std * min_coef)
                # print("Min TH\t" + str(min_th) + "\t" + "MAX TH\t" + str(max_th))
                # profit = asset + position * price
                # print("Profit\t" + str(profit) + "\t" + "Asset\t" + str(asset) + "\t" + "Position\t" + str(position))
                        
                    
                if is_update:
                    profit = asset + position * price
                    if profit > max_profit:
                        result_max_coef = max_coef
                        result_min_coef = min_coef
                        max_profit = profit

    print("Date\t" + str(date) + "\t"
             + "Max Profit\t" + str(max_profit) + "\t"
             + "Max Coef\t" + str(result_max_coef) + "\t"
             + "Min Coef\t" + str(result_min_coef))
    return result_max_coef, result_min_coef
            
with open(OUTPUT_FILE, "w") as ofp, open(DATA, "r") as ifp:
    writer = csv.writer(ofp, delimiter = "\t", lineterminator = "\n")
    reader = csv.reader(ifp, delimiter = "\t")
    mavg_data = np.array([])
    std_data = np.array([])
    eval_data = []
    coef_range = np.arange(0, COEF_WIDTH, COEF_STEP)
    prev_date = None
    for v in reader:
        skip_flag = False
        date = dt.strptime(v[DATE_IDX], '%Y-%m-%d %H:%M:%S')
        price = float(v[PRICE_IDX])
        if len(mavg_data) >= MOV_AVG_SPAN:
            mavg_data = np.hstack((mavg_data[1:], np.array(price))) 
        else:
            mavg_data = np.hstack((mavg_data, price))
            skip_flag = True
        if len(std_data) >= STD_SPAN:
            std_data = np.hstack((std_data[1:], np.array(price))) 
        else:
            std_data = np.hstack((std_data, price))        
            skip_flag = True
        if skip_flag:
            continue
        mavg = np.average(mavg_data)
        std = np.std(std_data)
        if is_calc_coef(prev_date, date):
            max_coef, min_coef = calculate_coef(eval_data, coef_range)
            writer.writerow([date, min_coef, max_coef])
            eval_data = []
        eval_data.append({"date": date, "price": price, "mavg": mavg, "std": std})
        prev_date = date
            

print("Success")
        


In [ ]:
""""
修正BB係数算出

入力データ
　フォーマット：TSV
　形式　　　　：日時(yyyy-mm-dd hh:MM:ss), 価格
 
閾値データ
　フォーマット：TSV
　形式　　　　：日時(yyyy-mm-dd hh:MM:ss), 下限の閾値, 上限の閾値

""""

import numpy as np
import csv
import datetime

DATA = "./data.tsv"
TH_FILE = "./threshold.tsv"

with open(TH_FILE, "r") as th_fp, open(DATA, "r") as data_fp:
    reader = csv.reader(th_fp, delimiter = "\t")
    coef_dict = {}
    for d in reader:
        date = datetime.datetime.strptime(d[0], '%Y-%m-%d %H:%M:%S')
        min_coef = float(d[1])
        max_coef = float(d[2])
        coef_dict[datetime.datetime(date.year, date.month, date.day)] = {"min": min_coef, "max": max_coef}
    
    # print(coef_dict)
    
    reader = csv.reader(data_fp, delimiter = "\t")
    asset = 0.0
    position = 0.0
    for d in reader:
        date = datetime.datetime.strptime(d[0], '%Y-%m-%d %H:%M:%S')
        price = float(d[1])
        coef_date = datetime.datetime(date.year, date.month, date.day) - datetime.timedelta(days = 1)
        if not coef_date in coef_dict:
            continue
        if price > coef_dict[coef_date]["max"]:
            if asset > price:
                new_position = float(asset // price)
                position += new_position
                asset -= new_position * price
            else:
                position += 1
                asset -= price
        elif price < coef_dict[coef_date]["min"]:
            if position > 0:
                asset += position * price
                position = 0
            else:
                position -= 1
                asset += price
    
    # print("Asset\t" + str(asset) + "\t" + "Position\t" + str(position))

    asset += position * price
    print("Asset\t" + str(asset))
    